In [81]:

import numpy as np
from sklearn.preprocessing import MinMaxScaler, RobustScaler
scaler = MinMaxScaler()
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN, LSTM, TimeDistributed, Input, Dropout, GRU
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
from keras.saving import register_keras_serializable
from sklearn.metrics import r2_score

def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(0, len(dataset)-2 *look_back+1, look_back):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back: i+ 2*look_back])
    return np.array(dataX), np.array(dataY)

def exponential_moving_average(data, span):
    return data.ewm(span=span, adjust=False).mean()

def read_data(file_path, num_features = 1):
    from pandas import read_csv
    series_influ_A_df = read_csv(file_path, index_col=0, engine='python')
    series_influ_A_df = series_influ_A_df.rename(columns= {"Influenza A - All types of surveillance": "case"})
    series_influ_A_df = series_influ_A_df[["case", "humidity", "temp", "dew","windspeed", "tempmax",][:num_features]]
    return series_influ_A_df.dropna()

def prepare_data(series, look_back, scaler, is_ema = False):
    if is_ema:
        span = 52  # Bạn có thể điều chỉnh độ dài span tùy ý
        series['case'] = exponential_moving_average(series['case'], span)
    series = series.astype('float32')
    series = series.values
    if scaler is not None:
        flattened_dataset = series.flatten()
        dataset = scaler.fit_transform(flattened_dataset.reshape(-1,1))
        dataset = dataset.reshape(series.shape)

    else: 
        dataset = series

    rest = len(dataset) % look_back
    dataset = dataset[rest:, :]
    trainsize = len(dataset) - look_back
    train = dataset[:trainsize, :]
    test = dataset[trainsize - look_back:, :]

    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    return trainX, trainY, testX, testY

def forecast(input, model):
    predicted = model.predict(input, verbose=0)
    return predicted


def save_plot(x,y, file_path):
    import numpy as np
    import matplotlib.pyplot as plt
    from scipy import stats

    # Generate some sample data
    # x = y_inverse.flatten()
    # y = y_hat_inverse.flatten()

    # Compute the linear regression line
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

    # Create the R-squared line
    r2_line = slope * x + intercept
    r2 = r2_score(x, y)
    r2_pearson = r_value**2

    # Create the scatter plot
    plt.figure(figsize=(8, 6))
    plt.scatter(x, y, label='Data Points')
    plt.plot(x, r2_line, color='red', label=f'R-squared line (R²={r2:.2f})')

    # Add labels and title
    plt.xlabel('actual number of infection')
    plt.ylabel('forecast number of infection')
    plt.title('Scatter Plot with R-squared Line')
    plt.legend()
    plt.grid()
    plt.savefig(file_path)
    plt.close()

def inverse_transform(data, scaler):
    flattened_data = data.flatten()
    inverse_flattened_data = scaler.inverse_transform(flattened_data.reshape(-1,1))
    return inverse_flattened_data.reshape(data.shape)

In [82]:
@register_keras_serializable()
class MyLSTM (Sequential):
    def __init__(self, look_back, dense_units =[],unit=64, optimizer='adam',name='lstm'):
        super().__init__(name=name)
        self.look_back = look_back
        self.add(Input(shape=(look_back,1)))
        self.add(LSTM(units=unit, activation='relu', return_sequences=True))
        for unit in dense_units:
            self.add(Dense(units=unit, activation='relu'))
        self.add(TimeDistributed(Dense(units=5, activation='sigmoid' )))
        self.compile(optimizer=optimizer, loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()])
    

def build_model(input_shape, dropout=None, dense_units = [], unit=64, optimizer='adam'):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(LSTM(units=unit, activation='relu', return_sequences=True))

    # if first_additional_layer:
    #     model.add(LSTM(units=unit, return_sequences=True))
    #     model.add(Dropout(dropout))

    # if second_additional_layer:
    #     model.add(LSTM(units=unit, return_sequences=True))
    #     model.add(Dropout(dropout))

    # if third_additional_layer:
    #     model.add(GRU(units=unit, return_sequences=True))
    #     model.add(Dropout(dropout))
    for unit in dense_units:
        model.add(Dense(units=unit, activation='relu'))
    model.add(TimeDistributed(Dense(units=input_shape[1], activation='sigmoid' )))
    model.compile(optimizer=optimizer, loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()])
    return model


In [83]:
# y_hat_inverse = np.expand_dims(scaler.inverse_transform(testY_hat[0]), axis=0)
# y_inverse = np.expand_dims(scaler.inverse_transform(testY[0]), axis=0)


In [84]:
def plot(testY, forecasts):
    import matplotlib.pyplot as plt
    forecastsPlot = forecasts[:,:,0].reshape(-1)
    testPlot = testY[:,:,0].reshape(-1)
    plt.plot(testPlot, "-y", label="actual", marker= '.')
    plt.plot(forecastsPlot, color = 'green', label="forecast")
    plt.ylabel("Number of infections")
    plt.legend(["actual", "forecast"])
    plt.show()


In [85]:
# plot(y_inverse, y_hat_inverse)
# print(model.predict(trainX).shape)
# print(trainY.shape)
# forecasts = model.predict(trainX)
# plot(trainY, forecasts)

In [86]:
import itertools
import tensorflow as tf
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
import os, json
from keras.models import load_model


def LSTM_HyperParameter_Tuning(config, df, scaler):
    
    n_neurons, n_batch_sizes, dropouts, look_backs, is_emas = config
    possible_combinations = list(itertools.product(n_neurons, n_batch_sizes, dropouts, look_backs, is_emas))
    
    print(possible_combinations)
    print('\n')
    
    hist = []
    for i in range(0, len(possible_combinations)):
        print(f'{i+1}th combination: \n')
        print('--------------------------------------------------------------------')
        n_neurons, n_batch_size, dropout, look_back, is_ema = possible_combinations[i]

        df = read_data('../temp_data/influA_vietnam_last_10_days.csv',num_features=2)
        
        trainX, trainY, testX, testY = prepare_data(df, look_back, scaler, is_ema=is_ema)
        model = build_model(input_shape=(trainX.shape[1], trainX.shape[2]), dense_units=n_neurons[1:], unit=n_neurons[0])

        es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=5)
        '''''
        From the mentioned article above --> If a validation dataset is specified to the fit() function via the validation_data or v
        alidation_split arguments,then the loss on the validation dataset will be made available via the name “val_loss.”
        '''''
        a = 'ema' if is_ema else 'not_ema'
        lstm_dir = os.path.join("../result", f"""lstm_{a}""")
        lstm_combination_dir = os.path.join(lstm_dir, str(i))
        os.makedirs(lstm_combination_dir, exist_ok=True)

        file_path = os.path.join(lstm_combination_dir, 'best_lstm_m2m_model.keras')

        mc = ModelCheckpoint(file_path, monitor='loss', mode='min', verbose=1, save_best_only=True)

        '''''
        cb = Callback(...)  # First, callbacks must be instantiated.
        cb_list = [cb, ...]  # Then, one or more callbacks that you intend to use must be added to a Python list.
        model.fit(..., callbacks=cb_list)  # Finally, the list of callbacks is provided to the callback argument when fitting the model.
        '''''

        model.fit(trainX, trainY,batch_size=n_batch_size, callbacks=[es, mc], verbose=0, epochs=200)
        train_accuracy = model.evaluate(trainX, trainY, verbose=0)
        # test_accuracy = model.evaluate(testX, testY, verbose=0)
        # hist.append(list((n_neurons, n_batch_size, dropout,look_back,
        #                   train_accuracy, test_accuracy)))
        hist.append(list((n_neurons, n_batch_size, dropout,look_back,
                          train_accuracy)))
        
        
        config= {
            "units": n_neurons,
            "n_batch_size": n_batch_size,
            "dropout": dropout,
            "look_back": look_back,
            "is_ema": is_ema
        }
        with open(os.path.join(lstm_combination_dir,'config.json'), 'w') as f:
            json.dump(config, f)

        

        # print(f'{str(i)}-th combination = {possible_combinations[i]} \n train accuracy: {train_accuracy}')
        
        # print('--------------------------------------------------------------------')
        # print('--------------------------------------------------------------------')
        # print('--------------------------------------------------------------------')
        # print('--------------------------------------------------------------------')
    for i in range(0, len(possible_combinations)):

        n_neurons, n_batch_size, dropout, look_back, is_ema = possible_combinations[i]
        df = read_data('../temp_data/influA_vietnam_last_10_days.csv',num_features=2)
        trainX, trainY, testX, testY = prepare_data(df, look_back, scaler, is_ema=is_ema)

        a = 'ema' if is_ema else 'not_ema'
        lstm_dir = os.path.join("../result", f"""lstm_{a}""")
        lstm_combination_dir = os.path.join(lstm_dir, str(i))
        os.makedirs(lstm_combination_dir, exist_ok=True)

        file_path = os.path.join(lstm_combination_dir, 'best_lstm_m2m_model.keras')


        model = load_model(file_path)

        #TODO: save r square
        testY_hat = forecast(testX, model)
        y_hat_inverse = inverse_transform(testY_hat, scaler)
        y_inverse = inverse_transform(testY, scaler)

        r2_image_path = os.path.join(lstm_combination_dir, 'r2_image.png')
        save_plot(y_inverse[:,:,0].flatten(), y_hat_inverse[:,:,0].flatten(), r2_image_path)
        
    import pandas as pd
    hist_df = pd.DataFrame(hist)
    hist_df = hist_df.rename(columns={0: 'units', 1: 'batch_size', 2: 'dropout', 3: 'look_back', 4: 'train_loss'})
    hist_df = hist_df.sort_values(by=['train_loss'], ascending=True)
    hist_df.to_csv(os.path.join(lstm_dir, 'history.csv'))
    return hist_df

In [87]:

#TODO: [units, batch_size, dropout, look_back, is_ema]
config = [[[64]], [1, 2, 4, 8], [0.2],[10,12,15,17], [True]] 
df = read_data('../temp_data/influA_vietnam_last_10_days.csv', num_features=2)
hist = LSTM_HyperParameter_Tuning(config, df, scaler)

[([64], 1, 0.2, 10, True), ([64], 1, 0.2, 12, True), ([64], 1, 0.2, 15, True), ([64], 1, 0.2, 17, True), ([64], 2, 0.2, 10, True), ([64], 2, 0.2, 12, True), ([64], 2, 0.2, 15, True), ([64], 2, 0.2, 17, True), ([64], 4, 0.2, 10, True), ([64], 4, 0.2, 12, True), ([64], 4, 0.2, 15, True), ([64], 4, 0.2, 17, True), ([64], 8, 0.2, 10, True), ([64], 8, 0.2, 12, True), ([64], 8, 0.2, 15, True), ([64], 8, 0.2, 17, True)]


1th combination: 

--------------------------------------------------------------------

Epoch 1: loss improved from inf to 0.07744, saving model to ../result\lstm_ema\0\best_lstm_m2m_model.keras

Epoch 2: loss improved from 0.07744 to 0.02754, saving model to ../result\lstm_ema\0\best_lstm_m2m_model.keras

Epoch 3: loss improved from 0.02754 to 0.01701, saving model to ../result\lstm_ema\0\best_lstm_m2m_model.keras

Epoch 4: loss improved from 0.01701 to 0.01224, saving model to ../result\lstm_ema\0\best_lstm_m2m_model.keras

Epoch 5: loss improved from 0.01224 to 0.00981, 

In [88]:
import pandas as pd
# hist = pd.DataFrame(hist)
# hist = hist.sort_values(by=[4], ascending=True)
hist

,units,batch_size,dropout,look_back,train_loss
12,[64],8,0.2,10,"[0.0035680029541254044, 0.05973276123404503]"
8,[64],4,0.2,10,"[0.0035725662019103765, 0.05977094918489456]"
4,[64],2,0.2,10,"[0.0035865954123437405, 0.059888191521167755]"
1,[64],1,0.2,12,"[0.003640269860625267, 0.06033464893698692]"
5,[64],2,0.2,12,"[0.0036754074972122908, 0.060625139623880386]"
2,[64],1,0.2,15,"[0.0036852953489869833, 0.06070663034915924]"
9,[64],4,0.2,12,"[0.0037550595588982105, 0.06127854064106941]"
10,[64],4,0.2,15,"[0.003757554106414318, 0.06129889190196991]"
6,[64],2,0.2,15,"[0.0037691318430006504, 0.06139325723052025]"
15,[64],8,0.2,17,"[0.003817929420620203, 0.06178940087556839]"


In [89]:
# def inverse_transform(data, scaler):
#     flattened_data = data.flatten()
#     inverse_flattened_data = scaler.inverse_transform(flattened_data.reshape(-1,1))
#     return inverse_flattened_data.reshape(data.shape)

In [90]:
# from keras.models import load_model

# model = load_model(r"D:\my_study\gr3\DATN\result\lstm_ema\10\best_lstm_m2m_model.keras")
# df = read_data('../temp_data/influA_vietnam_last_10_days.csv',num_features=2)
# trainX, trainY, testX, testY = prepare_data(df, 15, scaler, is_ema=True)
# testY_hat = forecast(testX, model)
# y_hat_inverse = inverse_transform(testY_hat, scaler)
# y_inverse = inverse_transform(testY, scaler)
# print(y_inverse, y_hat_inverse)

# save_plot(y_inverse[:,:,0].flatten(), y_hat_inverse[:,:,0].flatten(), 'abc.png')
# model.evaluate(testX, testY)

In [91]:
# hist.iloc[0][3]

In [92]:


# testY_hat = forecast(testX, model)
# y_hat_inverse = np.expand_dims(scaler.inverse_transform(testY_hat[0]), axis=0)
# y_inverse = np.expand_dims(scaler.inverse_transform(testY[0]), axis=0)

In [93]:


# testY_hat = forecast(testX, model)
# y_hat_inverse = inverse_transform(testY_hat, scaler)
# y_inverse = inverse_transform(testY, scaler)

In [94]:
# plot(y_inverse, y_hat_inverse)

In [95]:
# df = read_data('../temp_data/influA_vietnam_last_10_days.csv')
# trainX, trainY, testX, testY = prepare_data(df, hist.iloc[0][3], scaler=None, is_ema=True)
# testY